https://coredottoday.github.io/2018/09/17/%EB%AA%A8%EB%8D%B8-%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0-%ED%8A%9C%EB%8B%9D/

In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 138 (delta 26), reused 22 (delta 8), pack-reused 91
Receiving objects: 100% (138/138), 1.72 MiB | 4.55 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 44.2 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-11-19 06:58:45--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22cd:e0db
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response.

In [ ]:
import pandas as pd
import numpy as np
import warnings # 경고 메시지 무시
warnings.filterwarnings(action='ignore')
from konlpy.tag import Okt
okt = Okt()
from tqdm import tqdm # 작업 프로세스 시각화
import re # 문자열 처리를 위한 정규표현식 패키지
from gensim import corpora # 단어 빈도수 계산 패키지
from collections import Counter # 단어 등장 횟수 카운트

In [ ]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 18.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039191 sha256=d0c7141f4e20e4ec54da305b02f5a730b38aee1

In [ ]:
import json
import pickle
import glob, os
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
from bertopic import BERTopic

In [ ]:
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary

In [ ]:
dataset = pd.read_csv('/content/Market_kurly.csv')
dataset

,title,nickname,datetime,content
0,마켓컬리 8주년 위크 역대급 특가 추천 상품 포도 계란 소고기 마트보다 저렴하게!,고미리레,2023. 5. 22. 12:24,리뷰\n마켓컬리 8주년 위크 역대급 특가 추천 상품 포도 계란 소고기 마트보다 저렴...
1,마켓컬리 떡볶이 분식집 떡볶이 좋아하신다면 추억의 국민학교 떡볶이 드셔 보세요.,코나맘,2023. 5. 24. 23:38,식품 리뷰\n마켓컬리 떡볶이 분식집 떡볶이 좋아하신다면 추억의 국민학교 떡볶이 드셔...
2,자취생 재주문 마켓컬리 헤비유저의 실패하지 않는 밀키트 추천,영앤지,2023. 8. 29. 23:02,[블챌] 체크인 챌린지\n자취생 재주문 마켓컬리 헤비유저의 실패하지 않는 밀키트 추...
3,낭만어시장 땡초넣은 모듬 조개탕 마켓컬리,단감,2023. 8. 29. 13:29,"*내돈내산,직사용후기*\n낭만어시장 땡초넣은 모듬 조개탕 마켓컬리\n단감 ・ 202..."
4,마켓컬리 첫구매 신규가입 쿠폰 추천상품,행복바이러스,2023. 2. 24. 9:20,엄마의 장바구니\n마켓컬리 첫구매 신규가입 쿠폰 추천상품\n행복바이러스 ・ 2023...
...,...,...,...,...
3988,마켓컬리 컬리카드 장바구니혜택 챙겨요!,뷰스타 카민,2023. 5. 4. 12:00,리뷰\n마켓컬리 컬리카드 장바구니혜택 챙겨요!\n뷰스타 카민 ・ 2023. 5. 4...
3989,마켓컬리 추천상품 툭툭누들타이 사왓디만두,우리의 하루,2023. 1. 28. 9:00,레시피\n마켓컬리 추천상품 툭툭누들타이 사왓디만두\n우리의 하루 ・ 2023. 1....
3990,마켓컬리 오뗄블랙라벨 전지 베이컨/베이컨추천,호냥,2023. 6. 5. 14:10,일상스토리\n마켓컬리 오뗄블랙라벨 전지 베이컨/베이컨추천\n호냥 ・ 2023. 6....
3991,"쇼핑을 할땐 신규/첫구매 쿠폰을 활용하세요! (feat. 마켓컬리, 11번가 슈팅배...",각인,2023. 7. 21. 17:12,"정보/감상\n쇼핑을 할땐 신규/첫구매 쿠폰을 활용하세요! (feat. 마켓컬리, 1..."


In [ ]:
# txt 파일에서 stopword를 읽어오기
with open('/content/stopwords.txt', 'r', encoding='UTF-8') as file:
    stopword_data = file.read()


# 쉼표로 구분된 데이터를 리스트로 변환
stopword_list = stopword_data.split(',')
stopword_list = pd.DataFrame(stopword_list)
stopword_list.head()

,0
0,휴
1,아이구
2,아이쿠
3,아이고
4,어


In [ ]:
review_removed = list(map(lambda review: re.sub('[^가-힣 ]', '', review), dataset['content']))
dataset['content_removed'] = review_removed
dataset

,title,nickname,datetime,content,content_removed
0,마켓컬리 8주년 위크 역대급 특가 추천 상품 포도 계란 소고기 마트보다 저렴하게!,고미리레,2023. 5. 22. 12:24,리뷰\n마켓컬리 8주년 위크 역대급 특가 추천 상품 포도 계란 소고기 마트보다 저렴...,리뷰마켓컬리 주년 위크 역대급 특가 추천 상품 포도 계란 소고기 마트보다 저렴하게고...
1,마켓컬리 떡볶이 분식집 떡볶이 좋아하신다면 추억의 국민학교 떡볶이 드셔 보세요.,코나맘,2023. 5. 24. 23:38,식품 리뷰\n마켓컬리 떡볶이 분식집 떡볶이 좋아하신다면 추억의 국민학교 떡볶이 드셔...,식품 리뷰마켓컬리 떡볶이 분식집 떡볶이 좋아하신다면 추억의 국민학교 떡볶이 드셔 보...
2,자취생 재주문 마켓컬리 헤비유저의 실패하지 않는 밀키트 추천,영앤지,2023. 8. 29. 23:02,[블챌] 체크인 챌린지\n자취생 재주문 마켓컬리 헤비유저의 실패하지 않는 밀키트 추...,블챌 체크인 챌린지자취생 재주문 마켓컬리 헤비유저의 실패하지 않는 밀키트 추천영앤지...
3,낭만어시장 땡초넣은 모듬 조개탕 마켓컬리,단감,2023. 8. 29. 13:29,"*내돈내산,직사용후기*\n낭만어시장 땡초넣은 모듬 조개탕 마켓컬리\n단감 ・ 202...",내돈내산직사용후기낭만어시장 땡초넣은 모듬 조개탕 마켓컬리단감 복사 이웃추가...
4,마켓컬리 첫구매 신규가입 쿠폰 추천상품,행복바이러스,2023. 2. 24. 9:20,엄마의 장바구니\n마켓컬리 첫구매 신규가입 쿠폰 추천상품\n행복바이러스 ・ 2023...,엄마의 장바구니마켓컬리 첫구매 신규가입 쿠폰 추천상품행복바이러스 복사 이웃...
...,...,...,...,...,...
3988,마켓컬리 컬리카드 장바구니혜택 챙겨요!,뷰스타 카민,2023. 5. 4. 12:00,리뷰\n마켓컬리 컬리카드 장바구니혜택 챙겨요!\n뷰스타 카민 ・ 2023. 5. 4...,리뷰마켓컬리 컬리카드 장바구니혜택 챙겨요뷰스타 카민 복사 이웃추가본문 기타...
3989,마켓컬리 추천상품 툭툭누들타이 사왓디만두,우리의 하루,2023. 1. 28. 9:00,레시피\n마켓컬리 추천상품 툭툭누들타이 사왓디만두\n우리의 하루 ・ 2023. 1....,레시피마켓컬리 추천상품 툭툭누들타이 사왓디만두우리의 하루 복사 이웃추가본문...
3990,마켓컬리 오뗄블랙라벨 전지 베이컨/베이컨추천,호냥,2023. 6. 5. 14:10,일상스토리\n마켓컬리 오뗄블랙라벨 전지 베이컨/베이컨추천\n호냥 ・ 2023. 6....,일상스토리마켓컬리 오뗄블랙라벨 전지 베이컨베이컨추천호냥 복사 이웃추가본문 ...
3991,"쇼핑을 할땐 신규/첫구매 쿠폰을 활용하세요! (feat. 마켓컬리, 11번가 슈팅배...",각인,2023. 7. 21. 17:12,"정보/감상\n쇼핑을 할땐 신규/첫구매 쿠폰을 활용하세요! (feat. 마켓컬리, 1...",정보감상쇼핑을 할땐 신규첫구매 쿠폰을 활용하세요 마켓컬리 번가 슈팅배송 오늘의집각...


In [ ]:
review_tokenized = dataset['content_removed'].apply(lambda review: okt.nouns(review))

In [ ]:
def remove_stopword(tokens):
    review_removed_stopword = []
    for token in tokens:
        # 토큰의 글자 수가 2글자 이상인 경우
        if 1 < len(token):
            # 토큰이 불용어가 아닌 경우만 분석용 리뷰 데이터로 포함
            if token not in list(stopword_list[0]):
                review_removed_stopword.append(token)

    return review_removed_stopword

In [ ]:
dataset['content_removed_stopword'] = review_tokenized.apply(remove_stopword).replace(",", "")
content_removed_stopword = dataset['content_removed_stopword']

In [ ]:
dataset['content_removed_stopword'] = content_removed_stopword
dataset

,title,nickname,datetime,content,content_removed,content_removed_stopword,new_column
0,마켓컬리 8주년 위크 역대급 특가 추천 상품 포도 계란 소고기 마트보다 저렴하게!,고미리레,2023. 5. 22. 12:24,리뷰\n마켓컬리 8주년 위크 역대급 특가 추천 상품 포도 계란 소고기 마트보다 저렴...,리뷰마켓컬리 주년 위크 역대급 특가 추천 상품 포도 계란 소고기 마트보다 저렴하게고...,"[역대, 포도, 소고기, 역대, 포도, 소고기, 네이버, 플루, 애정, 벌써, 역대...",역대 포도 소고기 역대 포도 소고기 네이버 플루 애정 벌써 역대 진행 월요일 반값 ...
1,마켓컬리 떡볶이 분식집 떡볶이 좋아하신다면 추억의 국민학교 떡볶이 드셔 보세요.,코나맘,2023. 5. 24. 23:38,식품 리뷰\n마켓컬리 떡볶이 분식집 떡볶이 좋아하신다면 추억의 국민학교 떡볶이 드셔...,식품 리뷰마켓컬리 떡볶이 분식집 떡볶이 좋아하신다면 추억의 국민학교 떡볶이 드셔 보...,"[분식집, 추억, 국민학교, 요코, 능지, 추억, 국민학교, 추억, 국민학교, 냉동...",분식집 추억 국민학교 요코 능지 추억 국민학교 추억 국민학교 냉동식품 플라스틱 비닐...
2,자취생 재주문 마켓컬리 헤비유저의 실패하지 않는 밀키트 추천,영앤지,2023. 8. 29. 23:02,[블챌] 체크인 챌린지\n자취생 재주문 마켓컬리 헤비유저의 실패하지 않는 밀키트 추...,블챌 체크인 챌린지자취생 재주문 마켓컬리 헤비유저의 실패하지 않는 밀키트 추천영앤지...,"[자취, 헤비, 앤지, 위치, 역삼, 자취, 욥저, 재주, 편이, 자취, 식사, 해...",자취 헤비 앤지 위치 역삼 자취 욥저 재주 편이 자취 식사 해결 서울 레시피 대체로...
3,낭만어시장 땡초넣은 모듬 조개탕 마켓컬리,단감,2023. 8. 29. 13:29,"*내돈내산,직사용후기*\n낭만어시장 땡초넣은 모듬 조개탕 마켓컬리\n단감 ・ 202...",내돈내산직사용후기낭만어시장 땡초넣은 모듬 조개탕 마켓컬리단감 복사 이웃추가...,"[산직, 낭만, 어시장, 땡초, 모듬, 조개, 단감, 사랑, 드릴, 낭만, 어시장,...",산직 낭만 어시장 땡초 모듬 조개 단감 사랑 드릴 낭만 어시장 땡초 조개 땡초 모듬...
4,마켓컬리 첫구매 신규가입 쿠폰 추천상품,행복바이러스,2023. 2. 24. 9:20,엄마의 장바구니\n마켓컬리 첫구매 신규가입 쿠폰 추천상품\n행복바이러스 ・ 2023...,엄마의 장바구니마켓컬리 첫구매 신규가입 쿠폰 추천상품행복바이러스 복사 이웃...,"[신규가입, 바이러스, 신규가입, 충성, 신규가입, 삼겹살, 목란, 짜장면, 올반,...",신규가입 바이러스 신규가입 충성 신규가입 삼겹살 목란 짜장면 올반 핫도그 라이스 센...
...,...,...,...,...,...,...,...
3988,마켓컬리 컬리카드 장바구니혜택 챙겨요!,뷰스타 카민,2023. 5. 4. 12:00,리뷰\n마켓컬리 컬리카드 장바구니혜택 챙겨요!\n뷰스타 카민 ・ 2023. 5. 4...,리뷰마켓컬리 컬리카드 장바구니혜택 챙겨요뷰스타 카민 복사 이웃추가본문 기타...,"[카민, 평상시, 자꾸만, 자취, 면밀, 프로모션, 프로모션, 가득, 라면, 프로모...",카민 평상시 자꾸만 자취 면밀 프로모션 프로모션 가득 라면 프로모션 실질 프로모션 ...
3989,마켓컬리 추천상품 툭툭누들타이 사왓디만두,우리의 하루,2023. 1. 28. 9:00,레시피\n마켓컬리 추천상품 툭툭누들타이 사왓디만두\n우리의 하루 ・ 2023. 1....,레시피마켓컬리 추천상품 툭툭누들타이 사왓디만두우리의 하루 복사 이웃추가본문...,"[레시피, 툭툭, 타이, 왓디, 안녕하세요우, 서비스, 시스템, 일찍, 장점, 품질...",레시피 툭툭 타이 왓디 안녕하세요우 서비스 시스템 일찍 장점 품질 이색 만두 새우 ...
3990,마켓컬리 오뗄블랙라벨 전지 베이컨/베이컨추천,호냥,2023. 6. 5. 14:10,일상스토리\n마켓컬리 오뗄블랙라벨 전지 베이컨/베이컨추천\n호냥 ・ 2023. 6....,일상스토리마켓컬리 오뗄블랙라벨 전지 베이컨베이컨추천호냥 복사 이웃추가본문 ...,"[스토리, 블랙, 라벨, 전지, 베이컨, 베이컨, 블랙, 라벨, 시리즈, 전지, 베...",스토리 블랙 라벨 전지 베이컨 베이컨 블랙 라벨 시리즈 전지 베이컨 돼지 앞다리 베...
3991,"쇼핑을 할땐 신규/첫구매 쿠폰을 활용하세요! (feat. 마켓컬리, 11번가 슈팅배...",각인,2023. 7. 21. 17:12,"정보/감상\n쇼핑을 할땐 신규/첫구매 쿠폰을 활용하세요! (feat. 마켓컬리, 1...",정보감상쇼핑을 할땐 신규첫구매 쿠폰을 활용하세요 마켓컬리 번가 슈팅배송 오늘의집각...,"[감상, 활용, 번가, 슈팅, 각인, 네이버, 쿠팡, 신규가입, 최초, 활용, 소비...",감상 활용 번가 슈팅 각인 네이버 쿠팡 신규가입 최초 활용 소비자 커머스 번가 슈팅...


In [ ]:
# 각 내부 리스트의 모든 요소를 하나의 문자열로 합친 후 리스트 생성
output_list = [' '.join(item) for item in content_removed_stopword]

# 데이터프레임에 새로운 칼럼 추가
dataset['new_column'] = output_list
new_column = dataset['new_column']

In [ ]:
documents = list()
documents = new_column.values.tolist()
print(documents[:5])

['역대 포도 소고기 역대 포도 소고기 네이버 플루 애정 벌써 역대 진행 월요일 반값 나볼 일일 가도 기회 식탁 쟁여둘 수록 이득 쟁여템 퀄리티 원가 가기 장점 자재 채소 먹거리 주부 직장인 퀄리티 역대 규모 반값 반값 날씨 채소 칠레 포도 반값 반값 스틱 자외선 차단 스틱 대비 기회 일일 가일 월요일 가득 기회 정해진 개수 국탕류 평상시 놨으 전단 평상시 필수품 화장지 팬티 기저귀 율도 기회 단것 다이어터 페스 한정 반값 플렉스 플렉스 플렉스 주방 프리미엄 한정 롯데 현대 신한 하나카드 제휴 마눤 역대 보지 윤택 생활 바로가기 소비자 커머스 리세', '분식집 추억 국민학교 요코 능지 추억 국민학교 추억 국민학교 냉동식품 플라스틱 비닐 플라스틱 케이스 추억 국떡 밀떡 빨강 튀김 찰떡 찬물 요한 빨강 종이컵 가득 찰떡 식용유 스푼 달걀 라면 쫄면 국떡 요밀 찰떡 쫄깃해 두께 추억 분식집 라면 유부초밥 김말이 튀김 술안주 추억 국민학교 냉동식품 추억 분식집', '자취 헤비 앤지 위치 역삼 자취 욥저 재주 편이 자취 식사 해결 서울 레시피 대체로 평타 이상은 중앙 감속 우정 냉동식품 성수 최현석 셰프 중앙 감속 중앙 감속 구성 중앙 감속 튀김 가니쉬 구성 구가 자취 가니쉬 도로 분분 에프 후라이팬 발사믹 베이스 어간 가니쉬 중식 발사믹 베이스 양식 웍셔 너리 뱡뱡면 냉동식품 윅셔너리 뱡뱡면 구성 고추기름 구성 서면 고추기름 이쯤 들통 요리실 성품 중앙 감속 발사믹 뱡뱡면 편이 중식 호불호 갈릴 향신료 라면 금미 볶이 냉동식품 금미 옥닭 볶이 달달 마찬가지 볶이 구성 금미 볶이 욥닭 건강 밀떡 강경 밀떡파 금미 옥닭 볶이 한국인 라면 매콤 자취 편이 볶이 욥밥 정리 용태 자취 중앙 감속 발사믹 금미 옥닭 볶이 윅셔너리 뱡뱡면 뱡뱡면 볶이 금미', '산직 낭만 어시장 땡초 모듬 조개 단감 사랑 드릴 낭만 어시장 땡초 조개 땡초 모듬 조개 낭만 어시장 땡초 모듬 조개 일지 달렷 간단 낭만 어시장 땡초 모듬 조개 손질 구비 요밀 초가 수제비 낭만 어시장 땡초 모듬 조개 이

In [ ]:
len(documents)

3993

In [ ]:
vectorizer = CountVectorizer(max_features=100)

In [ ]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
		vectorizer_model=vectorizer,
        nr_topics=10, # 문서를 대표하는 토픽의 갯수
        top_n_words=50)

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

topics, probs = model.fit_transform(documents)

In [ ]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,45,-1_크리스탈_제이드_쌀국수_트러플,"[크리스탈, 제이드, 쌀국수, 트러플, 모나카, 핫도그, 그릭, 양배추, 돼지고기,...",[크리스탈 제이드 트러플 데왐마 크리스탈 제이드 트러플 지유 마넌 무배글 이동 그날...
1,0,12,0_쿠키_비건_세트_과자,"[쿠키, 비건, 세트, 과자, 초코, 케이크, 달달, 크기, 다이어트, 단백질, 식...",[맛보기 브레댄코 브라우니 쿠키 화이트데이 일치 쿠키 호불호 브라우니 쿠키 용이 백...
2,1,24,1_부대찌개_만두_카레_부산,"[부대찌개, 만두, 카레, 부산, 라면, 목란, 식사, 그릭, 편이, 냉장고, 오징...",[만두 장성 시장 만두 시장 부추 고기만 용도 발견 만두 장성 라면 만두 장성 시장...
3,2,24,2_간편식_초코_케이크_미니,"[간편식, 초코, 케이크, 미니, 냉동식품, 블루베리, 냉장고, 유기농, 제공, 과...",[간편식 티타임 무스 여유 한참 나이라 간편식 나볼 얼음 공장 냉각 기술 노하우 품...
4,3,3871,3_페스_그릭_다이어트_치킨,"[페스, 그릭, 다이어트, 치킨, 칼국수, 목란, 새우, 샌드위치, 레시피, 불고기...",[밍밍 퍼플 진행 신규가입 아이템 공개 퍼플 직전 목록 아이템 불고기 옹골 구성 고...
5,4,17,4_모나카_블루베리_트러플_초코,"[모나카, 블루베리, 트러플, 초코, 간편식, 케이크, 미니, 딸기, 활용, 냉동식...",[이즈 메르 모나카 유이 이즈 메르 모나카 발견 로켓 몽슈 모나카 작고 의미 모나카...
